In [21]:
# results printer
%load_ext autoreload
%autoreload 2

import torch

if torch.cuda.is_available():
    PROJECT_PATH = "/notebooks/evaluating-cmr-in-mm/"
else:
    PROJECT_PATH = "/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/"

print(PROJECT_PATH)
import sys
sys.path.append(PROJECT_PATH)

import os
%load_ext autoreload
%autoreload 2

if torch.cuda.is_available():
    os.chdir('/notebooks/evaluating-cmr-in-mm/')
else:
    if os.getcwd() != '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm':
        os.chdir('/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/notebooks/evaluating-cmr-in-mm/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from src.utils.utils import get_config, get_model
from src.data.dataset import Dataset
from src.utils.dataset_preprocessing import load_json_annotations
import pickle

model = 'clip'
dataset='f30k'

config = get_config(dataset=dataset, model=model)


json_file = load_json_annotations(config=config)

ds_test_split = Dataset(config=config, split="test", json_file=json_file)


def get_mean(dataf, col: str, round_factor=2):
    return round(dataf[col].describe().loc['mean'], round_factor)

results_root = os.path.join(
    config.results.dir,
    model,
    dataset
    )

filenames = os.listdir(results_root)

for file in filenames:
    if file.endswith('.pkl'):
        path = os.path.join(results_root, file)
        with open(path, 'rb') as f:
            data = pickle.load(f)
        if 'i2t' in file:
            task='i2t'
        elif 't2i' in file:
            task='t2i'
        else:
            print('Uknown task type')
        print('Loaded results dataframe from ', path)
        print('Df size: ', data.shape)
        print('Printing metrics...')
        print(
        f"""
        File: {path}
        Task: {task}
        R@1\tR@5\tR@10\tDCG
        {get_mean(data, f'{task}_recalls_at_1')}\t{get_mean(data, f'{task}_recalls_at_5')}\t{get_mean(data, f'{task}_recalls_at_10')}\t{get_mean(data, f'{task}_dcgs')}
        """
        )


Config path:  ./config/clip/f30k/development_dsp.yaml
Loaded config path
Loaded annotations from  /notebooks/evaluating-cmr-in-mm/datasets/f30k/annotations/dataset_flickr30k.json
Loaded results dataframe from  /notebooks/evaluating-cmr-in-mm/results/clip/f30k/i2t-results.pkl
Df size:  (1000, 8)
Printing metrics...

        File: /notebooks/evaluating-cmr-in-mm/results/clip/f30k/i2t-results.pkl
        Task: i2t
        R@1	R@5	R@10	DCG
        0.79	0.58	0.73	2.85
        
